**Celda 5.1 - Objetivo del Deployment**

## Paso 5 – Deployment (Despliegue)

**Objetivo:** Simular cómo se utilizaría el modelo entrenado en un entorno real.
Se cargan los artefactos (modelo y umbral) y se define una función para generar predicciones sobre nuevos clientes.

**Celda 5.2 - Cargar artefactos (modelo + umbral)**

In [1]:
import joblib
import pandas as pd
import numpy as np

model = joblib.load("../artifacts/log_reg_model.joblib")
threshold = joblib.load("../artifacts/decision_threshold.joblib")

print("Modelo y umbral cargados correctamente")
print("Umbral:", threshold)

Modelo y umbral cargados correctamente
Umbral: 0.3


**Celda 5.3 - Función de predicción**

In [4]:
def predict_default_risk(model, X_new, threshold=0.5):
    """
    Retorna:
    - prob_default: probabilidad de incumplimiento (clase 1)
    - pred: 0/1 según umbral
    """
    prob_default = model.predict_proba(X_new)[:, 1]
    pred = (prob_default >= threshold).astype(int)
    return prob_default, pred

**Celda 5.4 - Ejemplo práctico con un cliente del set de test**

In [3]:
X_test = pd.read_parquet("../artifacts/X_test.parquet")
y_test = pd.read_parquet("../artifacts/y_test.parquet")["TARGET"]

# Tomamos un ejemplo real (1 fila)
X_sample = X_test.iloc[[0]]
y_true = int(y_test.iloc[0])

prob, pred = predict_default_risk(model, X_sample, threshold=threshold)

print("Probabilidad de incumplimiento (clase 1):", float(prob[0]))
print("Predicción:", int(pred[0]), "(1=Incumple, 0=No incumple)")
print("Valor real:", y_true)

Probabilidad de incumplimiento (clase 1): 0.5386653962229693
Predicción: 1 (1=Incumple, 0=No incumple)
Valor real: 0


**Celda 5.5 - Predicción por lote (varios clientes)**

In [5]:
X_batch = X_test.sample(10, random_state=42)

prob_batch, pred_batch = predict_default_risk(model, X_batch, threshold=threshold)

result = X_batch.copy()
result["prob_incumplimiento"] = prob_batch
result["pred_incumple"] = pred_batch

result[["prob_incumplimiento", "pred_incumple"]].head(10)

,prob_incumplimiento,pred_incumple
14343,0.570409,1
94529,0.472375,1
26833,0.520859,1
101924,0.368956,1
281182,0.551305,1
208806,0.377843,1
190054,0.505063,1
259619,0.523172,1
292744,0.477806,1
225433,0.480680,1


**Celda 5.6 - Comentario final**

## Conclusión de Deployment

El modelo puede integrarse como un componente de scoring crediticio:
- Se carga el modelo entrenado y el umbral definido (artefactos).
- Se calcula la probabilidad de incumplimiento y se toma una decisión según el umbral.
- El umbral ajustado permite priorizar la detección de incumplimientos (mayor recall), asumiendo más falsos positivos.

**Recomendación:** Usar este modelo como apoyo a la decisión humana y monitorear métricas (recall, falsos positivos) periódicamente.